In [ ]:
'''
<d p="时间,模式,字体大小,颜色,时间戳,弹幕池,用户ID的CRC32b加密,弹幕ID">内容</d>
'''

In [55]:
client = pymongo.MongoClient("localhost", 27017)
db = client.mydb
db.collection_names()
books = db['book']
books.find_one()
for i in books.find():
    print i

# Now start to crawl the av numbers

In [11]:
import pymongo
client = pymongo.MongoClient("localhost", 27017) # Connect to the db server
db = client.bilibili
videos = db['videos']
videos2 = db['videos2']

In [5]:
import requests
from bs4 import BeautifulSoup
import re
p = re.compile(r'cid=(\d+)&')

def spider(url):
    flag = False
    VideoID = url.replace("http://bilibili.com/video/av", "")
    with requests.Session() as s:
        html = s.get(url)
        soup = BeautifulSoup(html.content,'html.parser')
    a = soup.find('div', class_ = 'usname')
    if a:
        flag = True
        
    if flag:
        # Poster ID and Poster Name
        posterid = a.a['mid']
        username = a.a['title']
        # The cataglog of video
        b = soup.find_all('span', typeof="v:Breadcrumb")
        cata1 = b[0].a.string
        cata2 = b[1].a.string
        # Title
        d = soup.find('div', class_ = 'v-title')
        title = d.string
        # Time
        e = soup.find('time')
        time = e.string
        # Cid
        f = soup.find('div', class_= 'scontent').script.string
        cid = str(p.findall(f)[0])
        # Video Info Click and Duration
        info_url = "http://interface.bilibili.com/player?id=cid:" + str(cid) + "&aid=" + str(VideoID)
        with requests.Session() as s:
            video_info = s.get(info_url)
            soup2 = BeautifulSoup(video_info.content,'html.parser')
        click = soup2.click.string
        duration = soup2.duration.string
        try:
            video = {'VideoID':str(VideoID), 'cid':cid, 'title':title, 'catalog1':cata1,'catalog2':cata2, 'time':time, 'click':click,
                    'duration':duration, 'posterid':posterid, 'postername':username}
            videos.insert_one(video)
            print "Succeed: av" + str(VideoID)
        except:
            print "Error"

In [10]:
urls = []
for i in range(1001, 8000000):
    url = 'http://bilibili.com/video/av' + str(i)
    urls.append(url)

In [12]:
# -*-coding:utf8-*-

from lxml import etree
from multiprocessing.dummy import Pool as ThreadPool
import requests
import time
import sys
import re
import json


reload(sys)

sys.setdefaultencoding('utf-8')

# id av cid title tminfo time click danmu coins favourites duration honor_click honor_coins honor_favourites
# mid name article fans tags[3] common


head = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36'
}

def spider(url):
    html = requests.get(url, headers=head)
    selector = etree.HTML(html.text)
    content = selector.xpath("//html")
    for each in content:
        title = each.xpath('//div[@class="v-title"]/h1/@title')
        if title:
            av = url.replace("http://bilibili.com/video/av", "")
            title = title[0]
            tminfo1_log = each.xpath('//div[@class="tminfo"]/a/text()')
            tminfo2_log = each.xpath('//div[@class="tminfo"]/span[1]/a/text()')
            tminfo3_log = each.xpath('//div[@class="tminfo"]/span[2]/a/text()')
            if tminfo1_log:
                tminfo1 = tminfo1_log[0]
            else:
                tminfo1 = ""
            if tminfo2_log:
                tminfo2 = tminfo2_log[0]
            else:
                tminfo2 = ""
            if tminfo3_log:
                tminfo3 = tminfo3_log[0]
            else:
                tminfo3 = ""
            tminfo = tminfo1 + '-' + tminfo2 + '-' + tminfo3
            time_log = each.xpath('//div[@class="tminfo"]/time/i/text()')
            mid_log = each.xpath('//div[@class="usname"]/a/@mid')
            username_log = each.xpath('//div[@class="usname"]/a/@title')
            if time_log:
                time = time_log[0]
            else:
                time = ""
            if mid_log:
                mid = mid_log[0]
            else:
                mid = ""
            if username_log:
                username = username_log[0]
            else:
                username = ""

            cid_html_1 = each.xpath('//div[@class="scontent"]/iframe/@src')
            cid_html_2 = each.xpath('//div[@class="scontent"]/script/text()')
            if cid_html_1 or cid_html_2:
                if cid_html_1:
                    cid_html = cid_html_1[0]
                else:
                    cid_html = cid_html_2[0]

                cids = re.findall(r'cid=.+&aid', cid_html)
                cid = cids[0].replace("cid=", "").replace("&aid", "")
                info_url = "http://interface.bilibili.com/player?id=cid:" + str(cid) + "&aid=" + av
                video_info = requests.get(info_url)
                video_selector = etree.HTML(video_info.text)
                for video_each in video_selector:
                    click_log = video_each.xpath('//click/text()')
                    duration_log = video_each.xpath('//duration/text()')
                    if click_log:
                        click = click_log[0]
                    else:
                        click = -1
                    if duration_log:
                        duration = duration_log[0]
                    else:
                        duration = ""
                try:

                    video = {'VideoID':str(av), 'cid':cid, 'title':title, 'catalog':tminfo, 'time':time, 'click':click,
                            'duration':duration, 'posterid':mid, 'postername':username}
                    videos2.insert_one(video)
                    print "Succeed: av" + str(av)
                except :
                    print "Error "
            else:
                print "Error_noCid:" + url
        else:
            print "Error_404: " + url


pool = ThreadPool(8)
# results = pool.map(spider, urls)
try:
    results = pool.map(spider, urls)
except Exception, e:
    # print 'ConnectionError'
    print e
    time.sleep(300)
    results = pool.map(spider, urls)

pool.close()
pool.join()

IndexError: list index out of range

In [ ]:
print 'done'